In [128]:
# !pip install matplotlib
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')

In [129]:
from itertools import chain

import nltk
nltk.download('conll2002')
import sklearn
import scipy.stats
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

[nltk_data] Downloading package conll2002 to /home/thais/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


### Let's use a rotulate data to build a NER system

In [130]:
df = pd.read_csv('result.csv')
result = pd.DataFrame(df)
result.tokenized_text = result.tokenized_text.apply(lambda x: x.encode('ascii','ignore'))

In [131]:
from sklearn.model_selection import train_test_split
train_sents, test_sents = train_test_split(df, train_size=0.8)

In [132]:
print(f"treino: {train_sents.shape}, teste: {test_sents.shape}")

treino: (934, 7), teste: (234, 7)


In [133]:
# %%time
# train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
# test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [134]:
train_sents['tokenized_text'][0]


b"[' O', ' O', ' O', ' O', ' B-NUM_AJUSTE', ' O', ' O', ' O', ' B-PROCESSO', ' O', ' O', ' O', ' B-CONTRATANTE', ' O', ' B-CONTRATADA', ' I-CONTRATADA', ' I-CONTRATADA', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' B-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' I-OBJ_AJUSTE', ' O', ' O', ' O', ' O', ' O', ' O', ' B-VALOR', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O

In [135]:
train_sents['treated_text'][0]


'EXTRATO DO CONTRATO Nº 292/2010. Processo: 150.001.430/2010. Partes: DF/SC/FAC X PATRÍCIA BARROSO HERZOG. Na qualidade de Beneficiário. Espécie: Contrato Nº 292/2010. Objeto: O presente contrato tem por objeto o fomento à produção e montagem mediante apoio financeiro do FAC ao Projeto “CAMINHOS DA MISSÃO CRULS” de interesse do Beneficiário acima especificado; no valor: R$ 221.733,00 (duzentos e vinte e um mil setecentos e trinta e três reais) que serão liberados em parcela única e transferidos à conta do Beneficiário, especialmente aberta no Banco de Brasília BRB, correndo a despesa à conta de dotação orçamentária própria do Programa de Trabalho 13.392.1300.9072/ 0002 Apoio à Arte e à Cultura do DF Fonte 100; Vigência: O presente contrato entrará em vigência a partir da liberação do recurso, com duração de 365 (trezentos e sessenta e cinco dias) dias. Executor: Fernanda da Costa Vieitas, matricula 174.389-9, p/ cedente: José Silvestre Gorgulho. P/ Beneficiário Patrícia Barroso Herzog,

### Features

Next, define some features. In this example we use word identity, word suffix, word shape and word POS tag; also, some information from nearby words is used.

This makes a simple baseline, but you certainly can add and remove some features to get (much?) better results - experiment with it.

sklearn-crfsuite (and python-crfsuite) supports several feature formats; here we use feature dicts.

In [136]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [138]:
sent2features(train_sents[0])[0]

TypeError: 'int' object is not subscriptable

In [ ]:
# %%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

### Training

To see all possible CRF parameters check its docstring. Here we are useing L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [ ]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

### Evaluation
There is much more O entities in data set, but we're more interested in other entities. To account for this we'll use averaged F1 score computed for all labels except for O. sklearn-crfsuite.metrics package provides some useful metrics for sequence classification task, including this one.

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
labels

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

In [ ]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

### Hyperparameter Optimization
To improve quality try to select regularization parameters using randomized search and 3-fold cross-validation.

I takes quite a lot of CPU time and RAM (we're fitting a model 50 * 3 = 150 times), so grab a tea and be patient, or reduce n_iter in RandomizedSearchCV, or fit model only on a subset of training data.

In [ ]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

### Check parameter space
A chart which shows which c1 and c2 values have RandomizedSearchCV checked. Red color means better results, blue means worse.

In [ ]:
_x = [s.parameters['c1'] for s in rs.grid_scores_]
_y = [s.parameters['c2'] for s in rs.grid_scores_]
_c = [s.mean_validation_score for s in rs.grid_scores_]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

### Check best estimator on our test data
As you can see, quality is improved.

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

### Let's check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])